# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [521]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Ask for parameters

In [526]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [527]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $t$: Time period (from 1 to 12)
*  $f$: Food product (1 for food 1, 2 for food 2)
*  $b$: Beverage product (1 for beverage 1, 2 for beverage 2, 3 for beverage 3)

## Parameters:

*  $FP_{ft}$: Fixed cost of producing one unit of food $f$ in period $t$ (from "fixed_cost_production.csv")
*  $VP_{ft}$: Variable cost of producing one unit of food  $f$ in period $t$ (from "variable_cost_production.csv")
*  $VB_{bt}$: Variable cost of producing one liter of beverage $b$ in period $t$ (from "variable_cost_production.csv")
*  $VS_{ft}$: Variable cost of storing one unit of food $f$ in period $t$ (from "variable_cost_storage.csv")
*  $VS_{bt}$: Variable cost of storing one liter of beverage $b$ in period $t$ (from "variable_cost_storage.csv")
*  $D_{ft}$: Maximum demand for food $f$ in period $t$ (from "demand.csv")
*  $D_{bt}$: Maximum demand for beverage $b$ in period $t$ (from "demand.csv")
*  $R_{ft}$: Revenue for selling one unit of food $f$ in period $t$ (from "revenue.csv")
*  $R_{bt}$: Revenue for selling one liter of beverage $b$ in period $t$ (from "revenue.csv")
*  $SC_f$: Storage capacity for food $f$ (580 for food 1, 687 for food 2)
*  $SC_b$: Storage capacity for beverage $b$ (599 for beverage 1, 788 for beverage 2, 294 for beverage 3)
*  $PC_f$: Production capacity for food $f$ (1080 for food 1, 908 for food 2)
*  $PC_b$: Production capacity for beverage $b$ (408 for beverage 1, 1000 for beverage 2, 403 for beverage 3) 

## Decision Variables:

*  $P_{ft}$: Quantity of food $f$ produced in period $t$ (integer)
*  $P_{bt}$: Quantity of beverage $b$ produced in period $t$ (continuous)
*  $S_{ft}$: Quantity of food $f$ stored at the end of period $t$ (integer)
*  $S_{bt}$: Quantity of beverage $b$ stored at the end of period $t$ (continuous)
*  $SL_{ft}$: Quantity of food $f$ sold in period $t$ (integer)
*  $SL_{bt}$: Quantity of beverage $b$ sold in period $t$ (continuous) 


# 2. Ask for objective

In [528]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [529]:
Markdown(response2.text)

## Objective Function:

Maximize 

$\sum_{t=1}^{12} \sum_{f=1}^{2} (R_{ft} \cdot SL_{ft} - FP_{ft} \cdot P_{ft} - VP_{ft} \cdot P_{ft} - VS_{ft} \cdot S_{ft}) + \sum_{t=1}^{12} \sum_{b=1}^{3} (R_{bt} \cdot SL_{bt} - VB_{bt} \cdot P_{bt} - VS_{bt} \cdot S_{bt})$ 


# 3. Ask for constraints

In [530]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [531]:
Markdown(response3.text)

## Constraints:

**1. Production Capacity:**

*   $P_{ft} \le PC_f$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $P_{bt} \le PC_b$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**2. Storage Capacity:**

*   $S_{ft} \le SC_f$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $S_{bt} \le SC_b$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**3. Demand Fulfillment:**

*   $SL_{ft} \le D_{ft}$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $SL_{bt} \le D_{bt}$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**4. Inventory Balance:**

*   $S_{f,t-1} + P_{ft} = SL_{ft} + S_{ft}$ for all $f = 1, 2$ and $t = 1, ..., 12$ (assuming $S_{f0} = 0$)
*   $S_{b,t-1} + P_{bt} = SL_{bt} + S_{bt}$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$ (assuming $S_{b0} = 0$)

**5. Non-Negativity:**

*   $P_{ft} \ge 0$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $P_{bt} \ge 0$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$
*   $S_{ft} \ge 0$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $S_{bt} \ge 0$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$
*   $SL_{ft} \ge 0$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $SL_{bt} \ge 0$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**6. Integer Constraints:**

* $P_{ft}, S_{ft}, SL_{ft} \in  \mathbb{Z}^+$ for all $f = 1, 2$ and $t=1,...,12$ 


## 3. Generate the pyomo code

In [534]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [535]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data from CSV files)
# Please provide the CSV files for a complete solution
fixed_cost_production = {
    (1, 1): 10,
    (1, 2): 12,
    # ... Add other data points
    (2, 12): 8,
}
variable_cost_production = {
    (1, 1): 5,
    (1, 2): 6,
    # ... Add other data points
    (3, 12): 4,
}
variable_cost_storage = {
    (1, 1): 2,
    (1, 2): 3,
    # ... Add other data points
    (3, 12): 1,
}
demand = {
    (1, 1): 100,
    (1, 2): 120,
    # ... Add other data points
    (3, 12): 80,
}
revenue = {
    (1, 1): 20,
    (1, 2): 24,
    # ... Add other data points
    (3, 12): 16,
}

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[1, 2, 3])  # Beverage products

# Define parameters
model.FP = pyo.Param(model.F, model.T, initialize=fixed_cost_production)
model.VP = pyo.Param(model.F, model.T, initialize=variable_cost_production)
model.VB = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.VS_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage)
model.VS_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)
model.SC_f = pyo.Param(model.F, initialize={1: 580, 2: 687})
model.SC_b = pyo.Param(model.B, initialize={1: 599, 2: 788, 3: 294})
model.PC_f = pyo.Param(model.F, initialize={1: 1080, 2: 908})
model.PC_b = pyo.Param(model.B, initialize={1: 408, 2: 1000, 3: 403})

# Define decision variables
model.P_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)  # Food production
model.P_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)     # Beverage production
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)  # Food storage
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)     # Beverage storage
model.SL_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers) # Food sold
model.SL_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)    # Beverage sold

# Define objective function
def objective_rule(model):
    return sum(
        model.R_f[f, t] * model.SL_f[f, t] - model.FP[f, t] * model.P_f[f, t] - 
        model.VP[f, t] * model.P_f[f, t] - model.VS_f[f, t] * model.S_f[f, t]
        for f in model.F for t in model.T
    ) + sum(
        model.R_b[b, t] * model.SL_b[b, t] - model.VB[b, t] * model.P_b[b, t] - 
        model.VS_b[b, t] * model.S_b[b, t]
        for b in model.B for t in model.T
    )
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def production_capacity_food_rule(model, f, t):
    return model.P_f[f, t] <= model.PC_f[f]
model.production_capacity_food = pyo.Constraint(model.F, model.T, rule=production_capacity_food_rule)

def production_capacity_beverage_rule(model, b, t):
    return model.P_b[b, t] <= model.PC_b[b]
model.production_capacity_beverage = pyo.Constraint(model.B, model.T, rule=production_capacity_beverage_rule)

def storage_capacity_food_rule(model, f, t):
    return model.S_f[f, t] <= model.SC_f[f]
model.storage_capacity_food = pyo.Constraint(model.F, model.T, rule=storage_capacity_food_rule)

def storage_capacity_beverage_rule(model, b, t):
    return model.S_b[b, t] <= model.SC_b[b]
model.storage_capacity_beverage = pyo.Constraint(model.B, model.T, rule=storage_capacity_beverage_rule)

def demand_fulfillment_food_rule(model, f, t):
    return model.SL_f[f, t] <= model.D_f[f, t]
model.demand_fulfillment_food = pyo.Constraint(model.F, model.T, rule=demand_fulfillment_food_rule)

def demand_fulfillment_beverage_rule(model, b, t):
    return model.SL_b[b, t] <= model.D_b[b, t]
model.demand_fulfillment_beverage = pyo.Constraint(model.B, model.T, rule=demand_fulfillment_beverage_rule)

def inventory_balance_food_rule(model, f, t):
    if t == 1:
        return model.P_f[f, t] == model.SL_f[f, t] + model.S_f[f, t]
    else:
        return model.S_f[f, t-1] + model.P_f[f, t] == model.SL_f[f, t] + model.S_f[f, t]
model.inventory_balance_food = pyo.Constraint(model.F, model.T, rule=inventory_balance_food_rule)

def inventory_balance_beverage_rule(model, b, t):
    if t == 1:
        return model.P_b[b, t] == model.SL_b[b, t] + model.S_b[b, t]
    else:
        return model.S_b[b, t-1] + model.P_b[b, t] == model.SL_b[b, t] + model.S_b[b, t]
model.inventory_balance_beverage = pyo.Constraint(model.B, model.T, rule=inventory_balance_beverage_rule)

# Solve the model
solver = pyo.SolverFactory('CPLEX')  # You can choose a different solver
results = solver.solve(model)

# Print the results
print(results)
print("Profit:", pyo.value(model.profit))

# Access the optimal values of decision variables
for t in model.T:
    for f in model.F:
        print(f"Food {f} Production in period {t}:", pyo.value(model.P_f[f, t]))
    for b in model.B:
        print(f"Beverage {b} Production in period {t}:", pyo.value(model.P_b[b, t]))
```

Remember to replace the sample data with your actual data from the CSV files. The structure provided allows you to load the data from appropriately named CSV files and use it within the Pyomo model. 


## 4. Input problem data and try running the generated code

In [548]:
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data from CSV files)
# Please provide the CSV files for a complete solution
fixed_cost_production = {
    (1, 1): 10,
    (1, 2): 12,
    # ... Add other data points
    (2, 12): 8,
}
variable_cost_production = {
    (1, 1): 5,
    (1, 2): 6,
    # ... Add other data points
    (3, 12): 4,
}
variable_cost_storage = {
    (1, 1): 2,
    (1, 2): 3,
    # ... Add other data points
    (3, 12): 1,
}
demand = {
    (1, 1): 100,
    (1, 2): 120,
    # ... Add other data points
    (3, 12): 80,
}
revenue = {
    (1, 1): 20,
    (1, 2): 24,
    # ... Add other data points
    (3, 12): 16,
}

##MANUAL START

fixed_cost_production = pd.read_csv("fixed_cost_production.csv", index_col=[0])
variable_cost_production = pd.read_csv("variable_cost_production.csv", index_col=[0])
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", index_col=[0])
demand = pd.read_csv("demand.csv", index_col=[0])
revenue = pd.read_csv("revenue.csv", index_col=[0])



# Function to convert DataFrame to the desired dictionary format
def convert_to_dict(df, item_type):
    if item_type == "food":
        data = df.iloc[:2, :]  # Select first two rows for food
    elif item_type == "beverage":
        data = df.iloc[2:, :]  # Select last three rows for beverage

    result_dict = {(i+1, t+1): data.iloc[i, t] for i in range(data.shape[0]) for t in range(data.shape[1])}
    return result_dict

# Convert each DataFrame to the desired dictionary format
fixed_cost_production_food = convert_to_dict(fixed_cost_production, "food")
fixed_cost_production_beverage = convert_to_dict(fixed_cost_production, "beverage")

variable_cost_production_food = convert_to_dict(variable_cost_production, "food")
variable_cost_production_beverage = convert_to_dict(variable_cost_production, "beverage")

variable_cost_storage_food = convert_to_dict(variable_cost_storage, "food")
variable_cost_storage_beverage = convert_to_dict(variable_cost_storage, "beverage")

demand_food = convert_to_dict(demand, "food")
demand_beverage = convert_to_dict(demand, "beverage")

revenue_food = convert_to_dict(revenue, "food")
revenue_beverage = convert_to_dict(revenue, "beverage")
###END



# Create a concrete model
model = pyo.ConcreteModel()

# Define sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[1,2, 3])  # Beverage products

# Define parameters
model.FP = pyo.Param(model.F, model.T, initialize=fixed_cost_production)
model.VP = pyo.Param(model.F, model.T, initialize=variable_cost_production)
model.VB = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.VS_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage)
model.VS_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)
model.SC_f = pyo.Param(model.F, initialize={1: 580, 2: 687})
model.SC_b = pyo.Param(model.B, initialize={1: 599, 2: 788, 3: 294})
model.PC_f = pyo.Param(model.F, initialize={1: 1080, 2: 908})
model.PC_b = pyo.Param(model.B, initialize={1: 408, 2: 1000, 3: 403})

# Define decision variables
model.P_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)  # Food production
model.P_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)     # Beverage production
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)  # Food storage
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)     # Beverage storage
model.SL_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers) # Food sold
model.SL_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)    # Beverage sold

# Define objective function
def objective_rule(model):
    return sum(
        model.R_f[f, t] * model.SL_f[f, t] - model.FP[f, t] * model.P_f[f, t] - 
        model.VP[f, t] * model.P_f[f, t] - model.VS_f[f, t] * model.S_f[f, t]
        for f in model.F for t in model.T
    ) + sum(
        model.R_b[b, t] * model.SL_b[b, t] - model.VB[b, t] * model.P_b[b, t] - 
        model.VS_b[b, t] * model.S_b[b, t]
        for b in model.B for t in model.T
    )
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def production_capacity_food_rule(model, f, t):
    return model.P_f[f, t] <= model.PC_f[f]
model.production_capacity_food = pyo.Constraint(model.F, model.T, rule=production_capacity_food_rule)

def production_capacity_beverage_rule(model, b, t):
    return model.P_b[b, t] <= model.PC_b[b]
model.production_capacity_beverage = pyo.Constraint(model.B, model.T, rule=production_capacity_beverage_rule)

def storage_capacity_food_rule(model, f, t):
    return model.S_f[f, t] <= model.SC_f[f]
model.storage_capacity_food = pyo.Constraint(model.F, model.T, rule=storage_capacity_food_rule)

def storage_capacity_beverage_rule(model, b, t):
    return model.S_b[b, t] <= model.SC_b[b]
model.storage_capacity_beverage = pyo.Constraint(model.B, model.T, rule=storage_capacity_beverage_rule)

def demand_fulfillment_food_rule(model, f, t):
    return model.SL_f[f, t] <= model.D_f[f, t]
model.demand_fulfillment_food = pyo.Constraint(model.F, model.T, rule=demand_fulfillment_food_rule)

def demand_fulfillment_beverage_rule(model, b, t):
    return model.SL_b[b, t] <= model.D_b[b, t]
model.demand_fulfillment_beverage = pyo.Constraint(model.B, model.T, rule=demand_fulfillment_beverage_rule)

def inventory_balance_food_rule(model, f, t):
    if t == 1:
        return model.P_f[f, t] == model.SL_f[f, t] + model.S_f[f, t]
    else:
        return model.S_f[f, t-1] + model.P_f[f, t] == model.SL_f[f, t] + model.S_f[f, t]
model.inventory_balance_food = pyo.Constraint(model.F, model.T, rule=inventory_balance_food_rule)

def inventory_balance_beverage_rule(model, b, t):
    if t == 1:
        return model.P_b[b, t] == model.SL_b[b, t] + model.S_b[b, t]
    else:
        return model.S_b[b, t-1] + model.P_b[b, t] == model.SL_b[b, t] + model.S_b[b, t]
model.inventory_balance_beverage = pyo.Constraint(model.B, model.T, rule=inventory_balance_beverage_rule)

# Solve the model
solver = pyo.SolverFactory('CPLEX')  # You can choose a different solver
results = solver.solve(model)

# Print the results
print(results)
print("Profit:", pyo.value(model.profit))

# Access the optimal values of decision variables
for t in model.T:
    for f in model.F:
        print(f"Food {f} Production in period {t}:", pyo.value(model.P_f[f, t]))
    for b in model.B:
        print(f"Beverage {b} Production in period {t}:", pyo.value(model.P_b[b, t]))

ValueError: Cannot construct DataFrameInitializer for DataFrame with multiple columns without also specifying the data column

## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [549]:
print(response.text)

## Indices:

*  $t$: Time period (from 1 to 12)
*  $f$: Food product (1 for food 1, 2 for food 2)
*  $b$: Beverage product (1 for beverage 1, 2 for beverage 2, 3 for beverage 3)

## Parameters:

*  $FP_{ft}$: Fixed cost of producing one unit of food $f$ in period $t$ (from "fixed_cost_production.csv")
*  $VP_{ft}$: Variable cost of producing one unit of food  $f$ in period $t$ (from "variable_cost_production.csv")
*  $VB_{bt}$: Variable cost of producing one liter of beverage $b$ in period $t$ (from "variable_cost_production.csv")
*  $VS_{ft}$: Variable cost of storing one unit of food $f$ in period $t$ (from "variable_cost_storage.csv")
*  $VS_{bt}$: Variable cost of storing one liter of beverage $b$ in period $t$ (from "variable_cost_storage.csv")
*  $D_{ft}$: Maximum demand for food $f$ in period $t$ (from "demand.csv")
*  $D_{bt}$: Maximum demand for beverage $b$ in period $t$ (from "demand.csv")
*  $R_{ft}$: Revenue for selling one unit of food $f$ in period $t$ (from "revenue.csv

In [550]:
print(response2.text)

## Objective Function:

Maximize 

$\sum_{t=1}^{12} \sum_{f=1}^{2} (R_{ft} \cdot SL_{ft} - FP_{ft} \cdot P_{ft} - VP_{ft} \cdot P_{ft} - VS_{ft} \cdot S_{ft}) + \sum_{t=1}^{12} \sum_{b=1}^{3} (R_{bt} \cdot SL_{bt} - VB_{bt} \cdot P_{bt} - VS_{bt} \cdot S_{bt})$ 



In [551]:
print(response3.text)

## Constraints:

**1. Production Capacity:**

*   $P_{ft} \le PC_f$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $P_{bt} \le PC_b$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**2. Storage Capacity:**

*   $S_{ft} \le SC_f$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $S_{bt} \le SC_b$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**3. Demand Fulfillment:**

*   $SL_{ft} \le D_{ft}$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $SL_{bt} \le D_{bt}$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$

**4. Inventory Balance:**

*   $S_{f,t-1} + P_{ft} = SL_{ft} + S_{ft}$ for all $f = 1, 2$ and $t = 1, ..., 12$ (assuming $S_{f0} = 0$)
*   $S_{b,t-1} + P_{bt} = SL_{bt} + S_{bt}$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$ (assuming $S_{b0} = 0$)

**5. Non-Negativity:**

*   $P_{ft} \ge 0$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $P_{bt} \ge 0$ for all $b = 1, 2, 3$ and $t = 1, ..., 12$
*   $S_{ft} \ge 0$ for all $f = 1, 2$ and $t = 1, ..., 12$
*   $S_{bt} \ge 0$ for all $b = 1, 2, 3$ and $t = 1, ..., 1

In [552]:
print(response4.text)

```python
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data from CSV files)
# Please provide the CSV files for a complete solution
fixed_cost_production = {
    (1, 1): 10,
    (1, 2): 12,
    # ... Add other data points
    (2, 12): 8,
}
variable_cost_production = {
    (1, 1): 5,
    (1, 2): 6,
    # ... Add other data points
    (3, 12): 4,
}
variable_cost_storage = {
    (1, 1): 2,
    (1, 2): 3,
    # ... Add other data points
    (3, 12): 1,
}
demand = {
    (1, 1): 100,
    (1, 2): 120,
    # ... Add other data points
    (3, 12): 80,
}
revenue = {
    (1, 1): 20,
    (1, 2): 24,
    # ... Add other data points
    (3, 12): 16,
}

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[1, 2, 3])  # Beverage products

# Define parameters
model.FP = pyo.Param(model.F, model.T, initialize=fixed_cost_pro